In [1]:
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import matplotlib.pyplot as plt
from tqdm import tqdm

root_dir = os.getcwd().split("AdversarialNIDS")[0] + "AdversarialNIDS"
sys.path.append(root_dir)

from scripts.logger import LoggerManager
from scripts.model_analyzer import perform_model_analysis

from CICIDS2017.preprocessing.dataset import CICIDS2017
from UNSWNB15.preprocessing.dataset import UNSWNB15

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
lm = LoggerManager(log_dir=f"{root_dir}/logs", log_name="test_dl_models")
lm.logger.info("Logger initialized")

dataset = UNSWNB15(logger=lm.logger).encode(attack_encoder="label").scale(scaler="minmax").optimize_memory()

2025-11-18 21:26:28,401 - INFO - Logger initialized
2025-11-18 21:26:28,403 - INFO - Downloading dataset: sweety18/cicids2017-full-dataset
2025-11-18 21:26:29,084 - INFO - Loading data
2025-11-18 21:26:42,599 - INFO - Initial dimensions: 2,214,469 rows x 79 columns = 174,943,051 cells
2025-11-18 21:26:57,557 - INFO - ============================================================
2025-11-18 21:26:57,557 - INFO - Preprocessing completed successfully
2025-11-18 21:26:57,558 - INFO - Final dimensions: 1,942,693 rows x 71 columns
2025-11-18 21:26:57,558 - INFO - Total rows removed: 271,776 (12.27%)
2025-11-18 21:26:57,559 - INFO - data retention rate: 87.73%
2025-11-18 21:26:57,559 - INFO - ============================================================
2025-11-18 21:26:57,561 - INFO - Encoding attack labels...
2025-11-18 21:26:57,990 - INFO - Attack labels encoded using LabelEncoder() encoder.
2025-11-18 21:26:57,991 - INFO - Scaling dataset features...
2025-11-18 21:27:24,188 - INFO - Features

In [3]:
X_train, X_val, y_train, y_val = dataset.split(
    multiclass=True,
    apply_smote=True,
    oneHot=True,
    toTensor=True
)

2025-11-18 21:27:24,887 - INFO - Splitting dataset into training and testing sets...
2025-11-18 21:28:11,055 - INFO - Class distribution after SMOTE:
2025-11-18 21:28:11,056 - INFO -   Class 0: 1222532 samples
2025-11-18 21:28:11,056 - INFO -   Class 1: 1222532 samples
2025-11-18 21:28:11,057 - INFO -   Class 2: 1222532 samples
2025-11-18 21:28:11,057 - INFO -   Class 3: 1222532 samples
2025-11-18 21:28:11,058 - INFO -   Class 4: 1222532 samples
2025-11-18 21:28:11,058 - INFO -   Class 5: 1222532 samples
2025-11-18 21:28:11,059 - INFO -   Class 6: 1222532 samples


In [4]:
def train_nids_model(model, optimizer, scheduler, criterion, train_loader, val_loader, device, epochs=25):
    epoch_losses = []
    epoch_val_losses = []
    # Training loop
    tqdm_epochs = tqdm(range(int(epochs)), desc="Training Progress")
    for epoch in tqdm_epochs:
        model.train()
        losses = []
        for X_train, y_train in train_loader:
            # Forward pass
            outputs = model(X_train)
            loss = criterion(outputs, y_train)
            losses.append(loss)

        epoch_loss = sum(losses) / len(losses)
        epoch_losses.append(epoch_loss.cpu().detach().numpy())
            
        # Backward pass and optimization
        optimizer.zero_grad()
        epoch_loss.backward()
        optimizer.step()

        scheduler.step(epoch_loss.item())
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_losses = []
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, y_val)
                val_losses.append(val_loss)

            epoch_val_loss = sum(val_losses) / len(val_losses)
            epoch_val_losses.append(epoch_val_loss.cpu().detach().numpy())
            
        tqdm_epochs.set_description(f"Loss: {epoch_loss.item():.4f}, Val Loss: {epoch_val_loss.item():.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

    return model, epoch_losses, epoch_val_losses


In [5]:
def display_loss(list_epoch_loss, list_val_loss, title, dir, logger, epoch_min=2):
    lm.logger.info("Plotting loss curve...")
    # Plotting loss curve with linear and log scale
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    plt.plot(list_epoch_loss[epoch_min:], label='Training Loss')
    plt.plot(list_val_loss[epoch_min:], '-r', label='Validation Loss')
    plt.title(f"Loss Curve - {title}")  
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.grid(True)
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(list_epoch_loss[epoch_min:], label='Training Loss')
    plt.plot(list_val_loss[epoch_min:], '-r', label='Validation Loss') 
    plt.xlabel('Epoch')
    plt.xscale('log')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.grid(True)
    plt.legend()
    loss_plot_path = f"{dir}/loss_img/{title}_loss.png"
    os.makedirs(f"{dir}/loss_img", exist_ok=True)
    plt.savefig(loss_plot_path, bbox_inches='tight', dpi=300)
    lm.logger.info(f"Loss curve saved as {loss_plot_path}")
    plt.show()

In [6]:
# Create DataLoaders
train_dataset = TensorDataset(X_train.to(device), y_train.to(device))
val_dataset = TensorDataset(X_val.to(device), y_val.to(device))

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
input_size = train_loader.dataset.tensors[0].shape[1]
num_classes = train_loader.dataset.tensors[1].shape[1]
print(f"Input size: {input_size}, Num classes: {num_classes}")  

criterion = nn.CrossEntropyLoss()

Input size: 70, Num classes: 7


In [8]:
class NetworkIntrusionMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetworkIntrusionMLP, self).__init__()

        self.features = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(32, 16),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(16, num_classes),
        )

    def forward(self, x):
        features = self.features(x)
        out = self.classifier(features)
        return torch.softmax(out, dim=1)

In [9]:
model_mlp = NetworkIntrusionMLP(input_size=input_size, num_classes=num_classes).to(device)

learning_rate_mlp = 1e-2
num_epochs_mlp = 50

optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=learning_rate_mlp)
scheduler_mlp = optim.lr_scheduler.ReduceLROnPlateau(optimizer_mlp, mode='min', factor=0.5, patience=5, min_lr=1e-5)

In [10]:
model_mlp, train_losses_mlp, val_losses_mlp = train_nids_model(
    model=model_mlp,
    optimizer=optimizer_mlp,
    scheduler=scheduler_mlp,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_mlp
)

Training Progress:   0%|          | 0/50 [00:39<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 7.67 GiB of which 2.06 MiB is free. Including non-PyTorch memory, this process has 7.66 GiB memory in use. Of the allocated memory 7.46 GiB is allocated by PyTorch, and 15.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
display_loss(
    list_epoch_loss=train_losses_mlp,
    list_val_loss=val_losses_mlp,
    title="MLP_NIDS",
    dir=root_dir,
    logger=lm.logger,
    epoch_min=2
)

In [ ]:
# Extract X and y from the validation dataset
#X_val = torch.stack([val_dataset[i][0] for i in range(len(val_dataset))]).to(device)
#y_val = torch.stack([val_dataset[i][1] for i in range(len(val_dataset))]).to(device)

perform_model_analysis(
    model=model_mlp,
    X_test=X_val,
    y_test=y_val,
    logger=lm.logger,
    model_name="MLP_NIDS",
    device=device
)

In [ ]:
class NetworkIntrustionCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(NetworkIntrustionCNN, self).__init__()
        
        self.features = nn.Sequential(
            # 1D Convolutional Layers
            nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128 * (input_size // 4), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        # Reshape input for 1D convolution
        x = x.unsqueeze(1)  # Add channel dimension
        features = self.features(x)
        features = features.view(features.size(0), -1)
        out = self.classifier(features)
        return torch.softmax(out, dim=1)

In [ ]:
model_cnn = NetworkIntrustionCNN(input_channels=1, num_classes=num_classes).to(device)

learning_rate_cnn = 1e-2
num_epochs_cnn = 100

optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=learning_rate_cnn)
scheduler_cnn = optim.lr_scheduler.PolynomialLR(optimizer_cnn, total_iters=num_epochs_cnn, power=0.9)

In [ ]:
model_cnn, train_loss_cnn, val_loss_cnn = train_nids_model(
    model=model_cnn,
    optimizer=optimizer_cnn,
    scheduler=scheduler_cnn,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_cnn
)

In [ ]:
display_loss(
    train_loss_cnn,
    val_loss_cnn,
    title="NetworkIntrustionCNN",
    dir=root_dir,
    logger=lm.logger,
    epoch_min=2
)

In [ ]:
perform_model_analysis(
    model=model_cnn
    X_test=X,
    y_test=y,
    logger=lm.logger,
    model_name="MLP_NIDS",
    device=device
)

In [ ]:
class NetworkIntrusionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(NetworkIntrusionLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # LSTM expects (batch, seq_len, features)
        lstm_out, _ = self.lstm(x)
        # Use the last time step
        out = lstm_out
        out = self.classifier(out)
        return torch.softmax(out, dim=1)

In [ ]:
model_lstm = NetworkIntrusionLSTM(input_size=input_size, hidden_size=128, num_layers=3, num_classes=num_classes).to(device)

learning_rate_lstm = 1e-2
num_epochs_lstm = 100

optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=learning_rate_lstm)
scheduler_lstm = optim.lr_scheduler.PolynomialLR(optimizer_lstm, total_iters=num_epochs_lstm, power=0.9)

In [ ]:
model_lstm, train_loss_lstm, val_loss_lstm = train_nids_model(
    model=model_lstm,
    optimizer=optimizer_lstm,
    scheduler=scheduler_lstm,
    criterion=criterion,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=num_epochs_lstm
)

In [ ]:
display_loss(
    train_loss_lstm, 
    val_loss_lstm, 
    title="LSTM_NIDS_Model", 
    dir=root_dir, 
    logger=lm.logger,
    epoch_min=2
)